# 4. Multiphase equilibrium with brine
<img style = "float: left;" src = "slides/3a_negative_flash.png" width = "80%">

## 4.1. Solubility of brine and CO$_2$ in different phases

In [ ]:
import numpy as np
import xarray as xr
from dartsflash.dartsflash import DARTSFlash
from dartsflash.components import CompData, ConcentrationUnits as cu
from dartsflash.mixtures import VLAq
from dartsflash.libflash import EoS, InitialGuess


comp_data = CompData(components=["H2O", "CO2"], ions=["Na+", "Cl-"])
f = VLAq(comp_data=comp_data, hybrid=True)

f.set_vl_eos("PR", root_order=[EoS.RootFlag.STABLE])
f.set_aq_eos("Aq")

f.init_flash(flash_type=DARTSFlash.FlashType.NegativeFlash, eos_order=["Aq", "VL"], nf_initial_guess=[InitialGuess.Henry_AV])

state_spec = {"pressure": np.arange(1., 600., 1.),
              "temperature": np.array([25, 90]) + 273.15,
             }
compositions = {"H2O": 0.8,
                "CO2": 0.2
               }
results_m0 = f.evaluate_flash(state_spec=state_spec, compositions=compositions,
                              mole_fractions=True, concentrations={"NaCl": 0.}, concentration_unit=cu.MOLALITY)
results_m2 = f.evaluate_flash(state_spec=state_spec, compositions=compositions,
                              mole_fractions=True, concentrations={"NaCl": 2.}, concentration_unit=cu.MOLALITY)
flash_results = xr.concat([results_m0, results_m2], dim='concentrations')


<img style = "float: left;" src = "slides/4a_dissol.png" width = "80%">

In [ ]:
from dartsflash.plot import PlotFlash
labels = [['25$\degree$C, m=0', '90$\degree$C, m=0'], ['25$\degree$C, m=2', '90$\degree$C, m=2']]

plotx = PlotFlash.solubility(f, flash_results, x_var='pressure', dissolved_comp_idx=1, phase_idx=0,
                             labels=labels, concentrations=[0, 2], styles=['solid', 'dashed'])

ploty = PlotFlash.solubility(f, flash_results, x_var='pressure', dissolved_comp_idx=0, phase_idx=1,
                             labels=labels, concentrations=[0, 2], styles=['solid', 'dashed'])
ploty.ax[0].set(ylim=[0., 0.04])

## 4.2. Hydrates formation

<img style = "float: left;" src = "slides/3c_vdwp.png" width = "60%">

In [ ]:
from dartsflash.hyflash import HyFlash
from dartsflash.components import CompData, ConcentrationUnits as cu
from dartsflash.mixtures import VLAq
from dartsflash.libflash import CubicEoS, AQEoS, Ballard, InitialGuess


comp_data = CompData(components=["H2O", "CO2"], ions=["Na+", "Cl-"], setprops=True)
f = HyFlash(comp_data=comp_data)

state_spec = {"temperature": np.arange(271., 291., 0.1),
              "pressure": None
             }
compositions = {"H2O": 0.8,
                "CO2": 0.2,
               }
concentrations = [{"NaCl": m} for m in [0., 0.01, 0.05, 0.10, 0.15]]  #, 0.20, 0.25]]
concentration_unit = cu.WEIGHT

labels = ["m = {:.0f} wt%".format(c["NaCl"]*100) for c in concentrations]

f.add_eos("VL", CubicEoS(comp_data, CubicEoS.SRK),
          trial_comps=[InitialGuess.Yi.Wilson])
f.add_eos("Aq", AQEoS(comp_data, {AQEoS.water: AQEoS.Jager2003,
                                      AQEoS.solute: AQEoS.Ziabakhsh2012,
                                      AQEoS.ion: AQEoS.Jager2003}))
f.add_hydrate_eos("sI", Ballard(comp_data, "sI"))

f.init_flash(flash_type=DARTSFlash.FlashType.NegativeFlash, eos_order=["Aq", "VL"], nf_initial_guess=[InitialGuess.Henry_AV])

if concentrations is not None:
    results = []
    for concentration in concentrations:
        results.append(f.evaluate_equilibrium(state_spec, compositions, mole_fractions=True,
                                              concentrations=concentration, concentration_unit=concentration_unit))
    results = xr.concat(results, dim='concentrations')
else:
    results = f.evaluate_equilibrium(state_spec, compositions, mole_fractions=True, print_state="")

In [ ]:
from dartsflash.plot import PlotHydrate
PlotHydrate.pt(f, results, compositions_to_plot=compositions, concentrations=concentrations, logy=True,
               labels=labels, legend_loc="lower right")